In [47]:
run ../src/wc_ratio_single_index.py

In [48]:
s = 3
ssy = SSY(L=s, K=s, I=s, J=s)

In [49]:
β, θ, N, H = ssy.β, ssy.θ, ssy.N, ssy.H
params = H, β, θ

In [50]:
w_init = np.ones(N)

In [51]:
out = T(w_init, params)

In [52]:
out.shape

(81,)

In [53]:
out

DeviceArray([1.99753103, 1.99779073, 1.9980505 , 1.99726803, 1.99779073,
             1.99831371, 1.99673883, 1.99779073, 1.99884375, 1.99751029,
             1.99776999, 1.99802975, 1.9972473 , 1.99776999, 1.99829295,
             1.99671811, 1.99776999, 1.99882298, 1.99734589, 1.99760554,
             1.99786526, 1.99708294, 1.99760554, 1.99812842, 1.99655384,
             1.99760554, 1.99865836, 1.99952412, 1.99978434, 2.00004463,
             1.99926059, 1.99978434, 2.00030836, 1.99873033, 1.99978434,
             2.00083945, 1.99950334, 1.99976355, 2.00002383, 1.99923982,
             1.99976355, 2.00028756, 1.99870957, 1.99976355, 2.00081864,
             1.99933861, 1.99959878, 1.99985902, 1.99907513, 1.99959878,
             2.0001227 , 1.99854497, 1.99959878, 2.0006537 , 2.00152119,
             2.00178193, 2.00204273, 2.00125713, 2.00178193, 2.00230699,
             2.00072582, 2.00178193, 2.00283915, 2.00150036, 2.0017611 ,
             2.0020219 , 2.00123631, 2.0017611 , 2.

In [54]:
run ../src/wc_ratio_multi_index.py

In [55]:
# Unpack                                                                                                                                                                                                                        
m = ssy                                                                                                                                                                                                                       
params = (m.L, m.K, m.I, m.J,                                                                                                                                                                                                   
         m.β, m.θ, m.γ, m.μ_c,                                                                                                                                                                                                      
         m.h_λ_states, m.h_λ_P,                                                                                                                                                                                                     
         m.h_c_states, m.h_c_P,                                                                                                                                                                                                     
         m.h_z_states, m.h_z_P,                                                                                                                                                                                                     
         m.z_states,   m.z_Q,                                                                                                                                                                                                       
         m.σ_c_states, m.σ_z_states)          

(L, K, I, J,                                                                                                                                                                                                                        
     β, θ, γ, μ_c,                                                                                                                                                                                                                      
     h_λ_states, h_λ_P,                                                                                                                                                                                                                 
     h_c_states, h_c_P,                                                                                                                                                                                                                 
     h_z_states, h_z_P,                                                                                                                                                                                                                 
     z_states,   z_Q,                                                                                                                                                                                                                   
     σ_c_states, σ_z_states) = params   

In [56]:
w_init = jnp.ones((m.L, m.K, m.I, m.J))

In [57]:
# Create intermediate arrays
B1 = jnp.exp(θ * h_λ_states)
B1 = jnp.reshape(B1, (1, 1, 1, 1, L, 1, 1, 1))

In [58]:
B2 = jnp.exp(0.5 * ((1-γ) * σ_c_states)**2)
B2 = jnp.reshape(B2, (1, K, 1, 1, 1, 1, 1, 1))

In [ ]:
B3 = jnp.exp((1-γ)*(μ_c + z_states))
B3 = jnp.reshape(B3, (1, 1, I, J, 1, 1, 1, 1))

In [59]:
# Reshape existing matrices prior to summing
Phλ = jnp.reshape(h_λ_P,  (L, 1, 1, 1, L, 1, 1, 1))
Phc = jnp.reshape(h_c_P,  (1, K, 1, 1, 1, K, 1, 1))
Phz = jnp.reshape(h_z_P,  (1, 1, I, 1, 1, 1, I, 1))
Pz  = jnp.reshape(z_Q,    (1, 1, I, J, 1, 1, 1, J))
w   = jnp.reshape(w_init, (1, 1, 1, 1, L, K, I, J))

# Take product and sum along last four axes
A = w * B1 * B2 * B3 * Phλ * Phc * Phz * Pz   

In [60]:
out = jnp.sum(A, axis=(4, 5, 6, 7))    

In [61]:
out.shape

(3, 3, 3, 3)

In [62]:
3**4

81

In [63]:
out

DeviceArray([[[[1.02253886, 1.01828253, 1.01404392],
               [1.02686854, 1.01828253, 1.00976831],
               [1.03563975, 1.01828253, 1.00121622]],

              [[1.02287961, 1.01862187, 1.01438185],
               [1.02721074, 1.01862187, 1.01010481],
               [1.03598487, 1.01862187, 1.00154987]],

              [[1.02558478, 1.02131577, 1.01706454],
               [1.02992736, 1.02131577, 1.01277619],
               [1.03872469, 1.02131577, 1.00419862]]],


             [[[0.99165089, 0.98752313, 0.98341256],
               [0.99584978, 0.98752313, 0.97926611],
               [1.00435604, 0.98752313, 0.97097235]],

              [[0.99198136, 0.98785222, 0.98374028],
               [0.99618165, 0.98785222, 0.97959244],
               [1.00469074, 0.98785222, 0.97129592]],

              [[0.9946048 , 0.99046475, 0.98634193],
               [0.99881621, 0.99046475, 0.98218313],
               [1.0073478 , 0.99046475, 0.97386466]]],


             [[[0.96169596, 0.